In [1]:
#import first to load and cleaning data
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

# for nicer plot fonts
sns.set(style="whitegrid")

In [2]:
data_path = "../data/insurance_claims.txt"
df = pd.read_csv(data_path, sep="|")

print("Shape of dataset:", df.shape)
print("\nFirst 5 rows:")
display(df.head())

print("\nData Types:")
print(df.dtypes)

print("\nMissing Values:")
print(df.isnull().sum())
print(df.columns)

Shape of dataset: (1000098, 52)

First 5 rows:


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0



Data Types:
UnderwrittenCoverID           int64
PolicyID                      int64
TransactionMonth             object
IsVATRegistered                bool
Citizenship                  object
LegalType                    object
Title                        object
Language                     object
Bank                         object
AccountType                  object
MaritalStatus                object
Gender                       object
Country                      object
Province                     object
PostalCode                    int64
MainCrestaZone               object
SubCrestaZone                object
ItemType                     object
mmcode                      float64
VehicleType                  object
RegistrationYear              int64
make                         object
Model                        object
Cylinders                   float64
cubiccapacity               float64
kilowatts                   float64
bodytype                     object
NumberOfDoors  

In [3]:
# Clean Gender Values
valid_gender_df = df[df['Gender'].isin(['Male', 'Female'])].copy()

# Create a new column indicating if a claim occurred
valid_gender_df['HasClaim'] = valid_gender_df['TotalClaims'] > 0

# Group by PolicyID using only cleaned data
policy_summary = valid_gender_df.groupby('PolicyID').agg({
    'TotalPremium': 'sum',
    'TotalClaims': 'sum',
    'HasClaim': 'max',
    'Province': 'first',
    'PostalCode': 'first',
    'Gender': 'first'
}).reset_index()

# Compute derived metrics
policy_summary['ClaimFrequency'] = policy_summary['HasClaim'].astype(int)
policy_summary['Margin'] = policy_summary['TotalPremium'] - policy_summary['TotalClaims']
policy_summary['ClaimSeverity'] = policy_summary.apply(
    lambda row: row['TotalClaims'] if row['HasClaim'] else 0, axis=1
)

# Sanity Check
print(policy_summary['Gender'].value_counts(dropna=False))
print(policy_summary.head())

# Summary stats
overall_claim_frequency = policy_summary['HasClaim'].mean()
avg_claim_severity = policy_summary[policy_summary['HasClaim']]['ClaimSeverity'].mean()
loss_ratio = policy_summary['TotalClaims'].sum() / policy_summary['TotalPremium'].sum()

print("Overall Claim Frequency:", overall_claim_frequency)
print("Average Claim Severity:", avg_claim_severity)
print("Loss Ratio:", loss_ratio)


Gender
Male      232
Female     35
Name: count, dtype: int64
   PolicyID  TotalPremium  TotalClaims  HasClaim      Province  PostalCode  \
0        14    915.785877          0.0     False  Western Cape        7530   
1        15    151.867402          0.0     False  Western Cape        7405   
2        16      0.000000          0.0     False  Western Cape        7888   
3        17   1692.981018          0.0     False  Western Cape        7785   
4        18      0.000000          0.0     False  Western Cape        7888   

   Gender  ClaimFrequency       Margin  ClaimSeverity  
0    Male               0   915.785877            0.0  
1  Female               0   151.867402            0.0  
2    Male               0     0.000000            0.0  
3    Male               0  1692.981018            0.0  
4  Female               0     0.000000            0.0  
Overall Claim Frequency: 0.2209737827715356
Average Claim Severity: 27914.407014570334
Loss Ratio: 0.873888151020368


In [4]:
# Data Segmentation & Hypothesis Tests

# Hypothesis 1: Risk differs by Province , ANOVA for ClaimFrequency, ClaimSeverity, and Margin
from scipy.stats import f_oneway

# Get province groups
grouped = policy_summary.groupby('Province')

# Claim Frequency
freq_groups = [group['ClaimFrequency'] for name, group in grouped]
severity_groups = [group['ClaimSeverity'] for name, group in grouped]
margin_groups = [group['Margin'] for name, group in grouped]

# Perform ANOVA
from scipy.stats import f_oneway
freq_p = f_oneway(*freq_groups).pvalue
severity_p = f_oneway(*severity_groups).pvalue
margin_p = f_oneway(*margin_groups).pvalue

print("Claim Frequency by Province p-value:", freq_p)
print("Claim Severity by Province p-value:", severity_p)
print("Margin by Province p-value:", margin_p)


Claim Frequency by Province p-value: 0.023671566753560237
Claim Severity by Province p-value: 0.14134633882595876
Margin by Province p-value: 0.16620840984424506


In [5]:
#Hypothesis 2 & 3: Zip Code (PostalCode) Risk / Margin Difference
from scipy.stats import f_oneway

# Get province groups
grouped = policy_summary.groupby('PostalCode')

# Claim Frequency
freq_groups = [group['ClaimFrequency'] for name, group in grouped]
severity_groups = [group['ClaimSeverity'] for name, group in grouped]
margin_groups = [group['Margin'] for name, group in grouped]

# Perform ANOVA
from scipy.stats import f_oneway
freq_p = f_oneway(*freq_groups).pvalue
severity_p = f_oneway(*severity_groups).pvalue
margin_p = f_oneway(*margin_groups).pvalue

print("Claim Frequency by PostalCode p-value:", freq_p)
print("Claim Severity by PostalCode p-value:", severity_p)
print("Margin by PostalCode p-value:", margin_p)


Claim Frequency by PostalCode p-value: 0.690846660745038
Claim Severity by PostalCode p-value: 0.7493752869527395
Margin by PostalCode p-value: 0.042491626356942945


In [6]:
#Hypothesis 4: Gender Difference , use t-test between Male and Female for the KPIs

from scipy.stats import chi2_contingency, ttest_ind

# Chi-squared test for Claim Frequency by Gender
contingency_table = pd.crosstab(policy_summary['Gender'], policy_summary['ClaimFrequency'])
chi2, p_freq, _, _ = chi2_contingency(contingency_table)

# T-test for Claim Severity by Gender
severity_male = policy_summary[(policy_summary['Gender'] == 'Male') & (policy_summary['HasClaim'])]['ClaimSeverity']
severity_female = policy_summary[(policy_summary['Gender'] == 'Female') & (policy_summary['HasClaim'])]['ClaimSeverity']
t_stat_sev, p_sev = ttest_ind(severity_male, severity_female, equal_var=False)

# T-test for Margin by Gender
margin_male = policy_summary[policy_summary['Gender'] == 'Male']['Margin']
margin_female = policy_summary[policy_summary['Gender'] == 'Female']['Margin']
t_stat_margin, p_margin = ttest_ind(margin_male, margin_female, equal_var=False)

# Output
print(f"Claim Frequency by Gender p-value: {p_freq}")
print(f"Claim Severity by Gender p-value: {p_sev}")
print(f"Margin by Gender p-value: {p_margin}")


Claim Frequency by Gender p-value: 0.44023619137780756
Claim Severity by Gender p-value: 0.787904953949884
Margin by Gender p-value: 0.8459898642167265


In [7]:
# Save the policy_summary DataFrame to CSV
policy_summary.to_csv("../eda_outputs/policy_summary.csv", index=False)